Here we are going to:
1.) investigate and analyse the datasets and then 
2.) transform the datasets, group them to convert them into a tabular format and then merge them together - so that we have a homogeneous grouping of risks which we can run our GLMs on. 
3.) We're going to run a frequency GLM and a severity GLM on the combined tabular dataset to produce a loss cost model (a model which predicts how much a particular grouping would cost given the claims experience). 4.) Lastly, we'll evaulate the model's performance.

Here we are going to apply GLM to model the loss cost of claims for the NFIP. I'm going to ignore adjusting the claims inflation, removing large losses and extreme events and just focus on applying the GLM on to a tabular form of the data.

Analaysis Questions:
* How many nulls are in the datasets?
* what are the differences between the datasets?
* How have the claims developed over time? - loss ratio/claims frequency and severity
* How are the claims distributed?

In [3]:
from pyspark.sql.functions import count, avg, sum
import pandas as pd


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

In [5]:
%sql

/* Query the created temp table in a SQL cell */

select * from `claims_csv`

_c0,agriculturestructureindicator,asofdate,basefloodelevation,basementenclosurecrawlspacetype,reportedcity,condominiumindicator,policycount,countycode,crsdiscount,dateofloss,elevatedbuildingindicator,elevationcertificateindicator,elevationdifference,censustract,floodzone,houseworship,latitude,locationofcontents,longitude,lowestadjacentgrade,lowestfloorelevation,numberoffloorsintheinsuredbuilding,nonprofitindicator,obstructiontype,occupancytype,originalconstructiondate,originalnbdate,amountpaidonbuildingclaim,amountpaidoncontentsclaim,amountpaidonincreasedcostofcomplianceclaim,postfirmconstructionindicator,ratemethod,smallbusinessindicatorbuilding,state,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,yearofloss,reportedzipcode,primaryresidence
93633,null,2019-05-31T00:00:00.000+0000,10.0,0,FAR ROCKAWAY,N,1,36081,0.0,2012-10-29T00:00:00.000+0000,N,3,0,36081101002,AE,null,40.6,null,-73.7,8.2,9.7,2,null,null,1,1986-05-01T00:00:00.000+0000,2012-08-01T00:00:00.000+0000,110068.9,null,0.0,Y,1,null,NY,250000,0,2012,11691,Y
93634,null,2019-05-31T00:00:00.000+0000,null,0,ISLAND PARK,N,1,36059,0.0,2012-10-29T00:00:00.000+0000,N,null,999,36059416201,X,null,40.6,Lowest floor above ground level and higher floors (No basement/enclosure/crawlspace/subgrade crawlspace),-73.6,null,null,2,null,null,1,1976-09-10T00:00:00.000+0000,2012-09-13T00:00:00.000+0000,112066.77,35246.02,0.0,N,Q,null,NY,250000,100000,2012,11558,Y
93635,null,2019-05-31T00:00:00.000+0000,null,2,POINT LOOKOUT,N,1,36059,0.0,2012-10-29T00:00:00.000+0000,N,null,999,36059416900,AE,null,40.6,Basement/Enclosure/Crawlspace/Subgrade Crawlspace and above,-73.6,null,null,2,null,null,1,1947-01-01T00:00:00.000+0000,2012-09-10T00:00:00.000+0000,21052.48,0.0,0.0,N,1,null,NY,250000,10500,2012,11569,Y
93636,null,2019-05-31T00:00:00.000+0000,null,2,FAR ROCKAWAY,N,1,36081,0.0,2012-10-28T00:00:00.000+0000,Y,3,999,36081095400,AE,null,40.6,null,-73.8,null,null,3,null,54,1,1960-04-01T00:00:00.000+0000,2012-08-31T00:00:00.000+0000,12463.61,null,0.0,N,1,null,NY,50000,0,2012,11692,Y
93637,null,2019-05-31T00:00:00.000+0000,null,0,LAWRENCE,N,1,36081,0.15,2012-10-29T00:00:00.000+0000,N,null,999,36059411600,AE,null,40.6,null,-73.7,null,null,2,null,null,1,1950-05-05T00:00:00.000+0000,2012-09-10T00:00:00.000+0000,55389.61,null,0.0,N,1,null,NY,250000,0,2012,11559,Y
93638,null,2019-05-31T00:00:00.000+0000,null,1,SHOREHAM,N,1,36103,0.0,2012-11-08T00:00:00.000+0000,N,null,999,36103158405,X,null,40.9,Basement/Enclosure/Crawlspace/Subgrade Crawlspace and above,-72.9,null,null,2,null,null,1,1968-08-01T00:00:00.000+0000,2012-09-05T00:00:00.000+0000,0.0,0.0,null,N,7,null,NY,250000,100000,2012,11786,Y
93639,null,2019-05-31T00:00:00.000+0000,null,2,SHIRLEY,N,1,36103,0.0,2012-10-29T00:00:00.000+0000,N,null,999,36103159505,AE,null,40.7,null,-72.9,null,null,2,null,null,1,1972-08-30T00:00:00.000+0000,2012-09-09T00:00:00.000+0000,0.0,null,null,N,1,null,NY,157300,0,2012,11967,Y
93640,null,2019-05-31T00:00:00.000+0000,8.0,4,FAR ROCKAWAY,N,1,36081,0.0,2012-10-29T00:00:00.000+0000,N,3,0,36081097203,AE,null,40.6,null,-73.8,8.7,8.2,2,null,null,1,1952-08-20T00:00:00.000+0000,2012-09-09T00:00:00.000+0000,72172.7,null,0.0,N,1,null,NY,250000,0,2012,11691,Y
93641,null,2019-05-31T00:00:00.000+0000,null,3,ISLAND PARK,N,1,36059,0.0,2012-10-29T00:00:00.000+0000,Y,null,999,36059416202,X,null,40.6,Basement/Enclosure/Crawlspace/Subgrade Crawlspace and above,-73.7,null,null,2,null,50,1,1930-09-10T00:00:00.000+0000,2012-09-11T00:00:00.000+0000,68446.05,52310.97,0.0,N,Q,null,NY,250000,100000,2012,11558,Y
93642,null,2019-05-31T00:00:00.000+0000,10.0,0,FAR ROCKAWAY,N,1,36081,0.0,2012-10-30T00:00:00.000+0000,N,3,2,36081099802,AE,null,40.6,null,-73.8,8.8,11.7,2,null,null,2,2008-05-14T00:00:00.000+0000,2012-09-04T00:00:00.000+0000,48949.84,null,0.0,Y,1,null,NY,250000,0,2012,11691,Y


In [6]:
claims = spark.sql('''
          select * from `claims_csv`
          '''
            )

In [7]:
claims

Out[ 4 ]: DataFrame[_c0: int, agriculturestructureindicator: string, asofdate: timestamp, basefloodelevation: double, basementenclosurecrawlspacetype: int, reportedcity: string, condominiumindicator: string, policycount: int, countycode: int, crsdiscount: double, dateofloss: timestamp, elevatedbuildingindicator: string, elevationcertificateindicator: string, elevationdifference: int, censustract: bigint, floodzone: string, houseworship: string, latitude: double, locationofcontents: string, longitude: double, lowestadjacentgrade: double, lowestfloorelevation: double, numberoffloorsintheinsuredbuilding: int, nonprofitindicator: string, obstructiontype: string, occupancytype: int, originalconstructiondate: timestamp, originalnbdate: timestamp, amountpaidonbuildingclaim: double, amountpaidoncontentsclaim: double, amountpaidonincreasedcostofcomplianceclaim: double, postfirmconstructionindicator: string, ratemethod: string, smallbusinessindicatorbuilding: string, state: string, totalbuildinginsurancecoverage: int, totalcontentsinsurancecoverage: int, yearofloss: int, reportedzipcode: string, primaryresidence: string]

How many nulls are in each column?

In [9]:
for col in claims.columns:
  print(col, "\t", "with null values: ", claims.filter(claims[col].isNull()).count())

_c0 with null values: 0
agriculturestructureindicator with null values: 168145
asofdate with null values: 0
basefloodelevation with null values: 153657
basementenclosurecrawlspacetype with null values: 2
reportedcity with null values: 234
condominiumindicator with null values: 8975
policycount with null values: 0
countycode with null values: 272
crsdiscount with null values: 0
dateofloss with null values: 0
elevatedbuildingindicator with null values: 8896
elevationcertificateindicator with null values: 106995
elevationdifference with null values: 0
censustract with null values: 1700
floodzone with null values: 27325
houseworship with null values: 167488
latitude with null values: 1659
locationofcontents with null values: 72913
longitude with null values: 1659
lowestadjacentgrade with null values: 157657
lowestfloorelevation with null values: 154115
numberoffloorsintheinsuredbuilding with null values: 303
nonprofitindicator with null values: 167465
obstructiontype with null values: 94564
occupancytype with null values: 76
originalconstructiondate with null values: 36329
originalnbdate with null values: 0
amountpaidonbuildingclaim with null values: 4794
amountpaidoncontentsclaim with null values: 58539
amountpaidonincreasedcostofcomplianceclaim with null values: 79375
postfirmconstructionindicator with null values: 9194
ratemethod with null values: 9003
smallbusinessindicatorbuilding with null values: 167209
state with null values: 0
totalbuildinginsurancecoverage with null values: 0
totalcontentsinsurancecoverage with null values: 0
yearofloss with null values: 0
reportedzipcode with null values: 82
primaryresidence with null values: 66199

In [10]:
claims_df = claims.toPandas()

In [11]:
claims_df.shape


Out[ 7 ]: (169312, 40)

In [12]:
claims_df.describe()

,_c0,basefloodelevation,basementenclosurecrawlspacetype,policycount,countycode,crsdiscount,elevationdifference,censustract,latitude,longitude,lowestadjacentgrade,lowestfloorelevation,numberoffloorsintheinsuredbuilding,occupancytype,amountpaidonbuildingclaim,amountpaidoncontentsclaim,amountpaidonincreasedcostofcomplianceclaim,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,yearofloss
count,169312.000000,15655.000000,169310.000000,169312.000000,169040.000000,169312.000000,169312.000000,1.676120e+05,167653.000000,167653.000000,11655.000000,15197.000000,169009.000000,169236.000000,1.645180e+05,110773.000000,89937.000000,1.693120e+05,169312.000000,169312.000000
mean,84655.500000,59.734155,1.132609,1.105391,36070.534087,0.008301,888.857878,3.604282e+10,40.996322,-74.015079,43.474809,166.892143,2.264578,1.339100,2.863880e+04,6210.707788,381.989360,1.610453e+05,27690.436590,2000.843998
std,48876.308726,195.766330,1.196702,3.929104,382.214499,0.029593,313.545474,1.075460e+09,0.743909,1.256027,227.629544,1064.500471,0.816117,0.813504,6.859566e+04,22111.813233,3251.298495,7.024136e+05,47061.344294,13.214408
min,0.000000,-999.000000,0.000000,1.000000,1003.000000,0.000000,-962.000000,3.600300e+04,28.000000,-98.000000,-14.500000,-999.000000,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,1974.000000
25%,42327.750000,7.000000,0.000000,1.000000,36059.000000,0.000000,999.000000,3.605941e+10,40.600000,-74.100000,3.900000,7.000000,2.000000,1.000000,5.426825e+02,0.000000,0.000000,3.300000e+04,0.000000,1989.000000
50%,84655.500000,8.000000,1.000000,1.000000,36081.000000,0.000000,999.000000,3.608109e+10,40.700000,-73.700000,6.700000,9.200000,2.000000,1.000000,6.551130e+03,334.500000,0.000000,1.560000e+05,6200.000000,2010.000000
75%,126983.250000,10.100000,2.000000,1.000000,36103.000000,0.000000,999.000000,3.610312e+10,41.000000,-73.500000,9.000000,12.000000,3.000000,1.000000,3.494000e+04,3718.650000,0.000000,2.500000e+05,37100.000000,2012.000000
max,169311.000000,6666.000000,4.000000,452.000000,54107.000000,0.300000,9998.000000,3.612315e+10,45.000000,-68.700000,9990.000000,9996.000000,6.000000,6.000000,9.467720e+06,500000.000000,45000.000000,1.095000e+08,1000000.000000,2019.000000


In [13]:
claims.select('amountpaidoncontentsclaim','amountpaidonbuildingclaim', 'totalcontentsinsurancecoverage',  'totalbuildinginsurancecoverage').describe().show()

+-------+-------------------------+-------------------------+------------------------------+------------------------------+
summary|amountpaidoncontentsclaim|amountpaidonbuildingclaim|totalcontentsinsurancecoverage|totalbuildinginsurancecoverage|
+-------+-------------------------+-------------------------+------------------------------+------------------------------+
 count| 110773| 164518| 169312| 169312|
 mean| 6210.707787818333| 28638.79994389663| 27690.43659043659| 161045.29094216594|
 stddev| 22111.813233265322| 68595.65584586072| 47061.344294209266| 702413.5911722986|
 min| 0.0| 0.0| 0| 0|
 max| 500000.0| 9467720.05| 1000000| 109500000|
+-------+-------------------------+-------------------------+------------------------------+------------------------------+

In [14]:
claims.select("lowestadjacentgrade",	"lowestfloorelevation",	"numberoffloorsintheinsuredbuilding",	"occupancytype", "countycode"	,"elevationdifference"	, "censustract", "basefloodelevation"	).show()

+-------------------+--------------------+----------------------------------+-------------+----------+-------------------+-----------+------------------+
lowestadjacentgrade|lowestfloorelevation|numberoffloorsintheinsuredbuilding|occupancytype|countycode|elevationdifference|censustract|basefloodelevation|
+-------------------+--------------------+----------------------------------+-------------+----------+-------------------+-----------+------------------+
 8.2| 9.7| 2| 1| 36081| 0|36081101002| 10.0|
 null| null| 2| 1| 36059| 999|36059416201| null|
 null| null| 2| 1| 36059| 999|36059416900| null|
 null| null| 3| 1| 36081| 999|36081095400| null|
 null| null| 2| 1| 36081| 999|36059411600| null|
 null| null| 2| 1| 36103| 999|36103158405| null|
 null| null| 2| 1| 36103| 999|36103159505| null|
 8.7| 8.2| 2| 1| 36081| 0|36081097203| 8.0|
 null| null| 2| 1| 36059| 999|36059416202| null|
 8.8| 11.7| 2| 2| 36081| 2|36081099802| 10.0|
 null| null| 2| 1| 36103| 999|36103124200| null|
 null| null| 3| 1| 36103| 999|36103169902| null|
 null| null| 2| 1| 36059| 999|36059413002| null|
 null| null| 4| 1| 36059| 999|36059416201| null|
 null| null| 3| 4| 36103| 999|36103110501| null|
 null| null| 4| 1| 36059| 999|36059416201| null|
 15.5| 10.2| 2| 1| 36081| 0|36081101002| 10.0|
 null| null| 2| 1| 36059| 999|36059416202| null|
 null| null| 3| 2| 36059| 999|36059416702| null|
 null| null| 3| 2| 36081| 999|36081096400| null|
+-------------------+--------------------+----------------------------------+-------------+----------+-------------------+-----------+------------------+
only showing top 20 rows

In [15]:
%sql

/* Query the created temp table in a SQL cell */

select date_format(originalnbdate, "y"), yearofloss from `claims_csv`

"date_format(originalnbdate, y)",yearofloss
2012,2012
2012,2012
2012,2012
2012,2012
2012,2012
2012,2012
2012,2012
2012,2012
2012,2012
2012,2012


How does the loss ratio develop over time?

In [17]:
%sql

/* Query the created temp table in a SQL cell */

select (SUM(totalbuildinginsurancecoverage) - SUM(amountpaidoncontentsclaim))/SUM(totalbuildinginsurancecoverage) as loss_ratio_building, 
       (SUM(totalcontentsinsurancecoverage) - SUM(amountpaidoncontentsclaim))/SUM(totalcontentsinsurancecoverage) as loss_ratio_content, 
       (SUM(totalcontentsinsurancecoverage + totalbuildinginsurancecoverage) - SUM(amountpaidoncontentsclaim + amountpaidoncontentsclaim))/SUM(totalcontentsinsurancecoverage + totalbuildinginsurancecoverage) as loss_ratio_total, 
       yearofloss
       from `claims_csv`
       group by yearofloss
       order by yearofloss desc

loss_ratio_building,loss_ratio_content,loss_ratio_total,yearofloss
0.9995118364096438,0.9980935540506647,0.9992227062625271,2019
0.9870662181677751,0.9505310972595422,0.9794938310777508,2018
0.9921799823245452,0.9659565544349022,0.9872814943893815,2017
0.9954552252528348,0.9833926723273383,0.9928634457297386,2016
0.988425330398741,0.9600039080963578,0.9820463558955883,2015
0.9954770668595936,0.9840951826967147,0.9929569900051963,2014
0.9913585401459853,0.9590637024651496,0.98572951297717,2013
0.9694862344542242,0.8057127397767581,0.9472561489726067,2012
0.9835538295532421,0.8983150712239317,0.9716869193375878,2011
0.9970442216535649,0.9874300593003691,0.9952138824069013,2010


How are the claims distributed?

In [19]:
%sql

/* Query the created temp table in a SQL cell */

select amountpaidoncontentsclaim from `claims_csv`


amountpaidoncontentsclaim
null
35246.02
0.0
null
null
0.0
null
null
52310.97
null


In [20]:
%sql

/* Query the created temp table in a SQL cell */

select amountpaidonbuildingclaim  from `claims_csv`

amountpaidonbuildingclaim
110068.9
112066.77
21052.48
12463.61
55389.61
0.0
0.0
72172.7
68446.05
48949.84


In [21]:
%sql
select totalbuildinginsurancecoverage  from `claims_csv`  


totalbuildinginsurancecoverage
250000
250000
250000
50000
250000
250000
157300
250000
250000
250000


In [22]:
%sql
select totalcontentsinsurancecoverage from `claims_csv` 

totalcontentsinsurancecoverage
0
100000
10500
0
0
100000
0
0
100000
0


In [23]:
%sql
select amountpaidoncontentsclaim/totalcontentsinsurancecoverage * 100 from `claims_csv` 

((amountpaidoncontentsclaim / CAST(totalcontentsinsurancecoverage AS DOUBLE)) * CAST(100 AS DOUBLE))
null
35.246019999999994
0.0
null
null
0.0
null
null
52.31097
null


In [24]:
%sql
select amountpaidonbuildingclaim/totalbuildinginsurancecoverage * 100 from `claims_csv` 

((amountpaidonbuildingclaim / CAST(totalbuildinginsurancecoverage AS DOUBLE)) * CAST(100 AS DOUBLE))
44.02756
44.826708
8.420992
24.92722
22.155844
0.0
0.0
28.869079999999997
27.378420000000002
19.579935999999996


In [25]:
%sql

/* Query the created temp table in a SQL cell */

select log10(amountpaidonbuildingclaim)  from `claims_csv` where amountpaidonbuildingclaim != 0

LOG10(amountpaidonbuildingclaim)
5.0416646262710385
5.049476854833603
4.323303263443607
4.095643850991368
4.7434283072762335
4.858372952674347
4.8353483901953265
4.689751276583882
4.499139884319948
4.562892965892898


In [26]:
%sql

/* Query the created temp table in a SQL cell */

select log10(amountpaidoncontentsclaim)  from `claims_csv` where amountpaidoncontentsclaim != 0

LOG10(amountpaidoncontentsclaim)
4.547110083323859
4.718592773205446
3.591327331383886
3.066612874965157
3.562086971972978
5.592126993949545
3.658579902444917
4.9146069862443005
4.4980471398066735
4.340197746458496


In [27]:
%sql

/* Query the created temp table in a SQL cell */

select log10(amountpaidoncontentsclaim + amountpaidonbuildingclaim)  from `claims_csv` where amountpaidoncontentsclaim != 0 and amountpaidonbuildingclaim != 0 

LOG10((amountpaidoncontentsclaim + amountpaidonbuildingclaim))
5.16824045482148
5.081912387114187
4.606948909394316
4.328025498919923
4.33824686605051
5.949855866907868
4.661579674670928
5.329979424090736
5.0050921426106845
5.180646372289837


Interestingly the dataset seems to reveal impact of Hurrican Sandy hitting New York in 2012 and the frequency & severity of that event

In [29]:
%sql
select count(*), cast(yearofloss as string) from `claims_csv` group by cast(yearofloss as string) order by cast(yearofloss as string)

count(1),yearofloss
1,1974
2,1975
12,1976
2358,1977
4267,1978
10305,1979
7324,1980
2548,1981
2209,1982
2830,1983


In [30]:
%sql
select sum(amountpaidoncontentsclaim + amountpaidonbuildingclaim), cast(yearofloss as string) from `claims_csv` group by cast(yearofloss as string) order by cast(yearofloss as string)

sum((amountpaidoncontentsclaim + amountpaidonbuildingclaim)),yearofloss
null,1974
3174.2799999999997,1975
72003.1,1976
3768556.960000002,1977
9025862.330000008,1978
2.1473386060000043E7,1979
1.1946320759999985E7,1980
4489439.099999998,1981
3075367.530000001,1982
4284077.35,1983


Almost all of the claims cost in the last 50 years have been generated by one major event - Hurricane Sandy.

Normally, you would try and model the extreme events like Hurricane's using a catastrophe model and you would bake that into your prices. However, I will just leave it in and model it as if it were not an extreme event.

In [32]:
%sql
select sum(amountpaidoncontentsclaim + amountpaidonbuildingclaim) from `claims_csv`

sum((amountpaidoncontentsclaim + amountpaidonbuildingclaim))
3.311941612490001E9


In [33]:
%sql

/* Query the created temp table in a SQL cell */

select * from `policies_parquet`

agriculturestructureindicator,basefloodelevation,basementenclosurecrawlspacetype,cancellationdateoffloodpolicy,censustract,condominiumindicator,construction,countycode,crsdiscount,deductibleamountinbuildingcoverage,deductibleamountincontentscoverage,elevatedbuildingindicator,elevationcertificateindicator,elevationdifference,federalpolicyfee,floodzone,hfiaasurcharge,houseofworshipindicator,latitude,locationofcontents,longitude,lowestadjacentgrade,lowestfloorelevation,nonprofitindicator,numberoffloorsininsuredbuilding,obstructiontype,occupancytype,originalconstructiondate,originalnbdate,policycost,policycount,policyeffectivedate,policyterminationdate,policytermindicator,postfirmconstructionindicator,primaryresidenceindicator,propertystate,reportedzipcode,ratemethod,regularemergencyprogramindicator,reportedcity,smallbusinessindicatorbuilding,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,totalinsurancepremiumofthepolicy
null,null,2,null,36017970400,N,N,36017,0.00,G,G,N,1,999,50,A04,25,null,42.5,Basement/Enclosure/Crawlspace/Subgrade Crawlspace and above,-75.5,null,null,null,3,null,1,1901-06-15,2010-06-04,1371,1,2016-06-04,2017-06-04,1,N,Y,NY,13815,1,R,NORWICH,null,87100,5600,1127
null,null,0,null,36059411100,N,N,36059,0.00,F,F,N,null,999,50,X,250,N,40.6,Lowest floor above ground level and higher floors (No basement/enclosure/crawlspace/subgrade crawlspace),-73.8,null,null,N,2,null,1,1960-10-15,2011-10-17,700,1,2016-10-17,2017-06-06,1,N,N,NY,11096,null,R,INWOOD,N,250000,100000,348
null,null,1,null,36059415402,N,N,36059,0.00,F,F,N,1,999,25,X,250,N,40.7,Basement/Enclosure/Crawlspace/Subgrade Crawlspace and above,-73.5,null,null,N,2,null,1,1964-01-01,2006-07-20,724,1,2016-07-20,2017-07-20,1,N,N,NY,11710,7,R,BELLMORE,N,250000,100000,390
null,null,0,null,36059415401,N,N,36059,0.00,2,2,N,1,999,45,A04,250,null,40.7,Lowest floor only above ground level (No basement/enclosure/crawlspace/subgrade crawlspace),-73.5,null,null,null,1,null,1,1955-08-15,1994-11-23,1082,1,2016-02-10,2017-02-10,1,N,N,NY,11710,1,R,BELLMORE,null,33600,10800,684
null,null,2,null,36119007200,N,N,36119,0.10,2,null,N,null,999,50,A08,250,N,40.9,null,-73.7,null,null,N,3,null,1,1957-09-01,2014-08-29,5489,1,2016-08-29,2017-08-29,1,N,N,NY,10543,1,R,MAMARONECK,N,250000,0,4512
null,null,0,null,36081091601,N,N,36081,0.00,5,null,N,null,999,50,AE,250,null,40.6,null,-73.9,null,null,null,1,null,1,1950-01-01,2014-07-18,2774,1,2016-07-18,2017-07-18,1,N,N,NY,11697,1,R,BREEZY POINT,N,167500,0,2151
N,6.0,0,null,36103123600,L,N,36103,0.00,A,0,N,null,0,400,AE,250,N,40.7,null,-73.4,5.6,6.3,N,2,null,3,1976-01-01,2015-09-02,6359,8,2016-09-02,2017-09-02,1,N,N,NY,11701,B,R,AMITYVILLE,N,1373900,0,4964
N,null,2,null,36103169902,N,N,36103,0.00,F,F,N,null,999,25,X,250,N,40.9,Basement/Enclosure/Crawlspace/Subgrade Crawlspace and above,-72.6,null,null,N,2,null,1,2000-03-02,2014-05-24,724,1,2016-05-24,2017-05-24,1,Y,N,NY,11970,7,R,SOUTH JAMESPORT,N,250000,100000,390
N,null,1,null,36061004700,N,N,36061,0.00,F,F,N,null,999,25,X,250,N,40.7,Basement/Enclosure/Crawlspace/Subgrade Crawlspace and above,-74.0,null,null,N,3,null,6,1974-12-30,2014-05-26,3375,1,2016-05-26,2017-05-26,1,N,N,NY,10012,7,R,NEW YORK,N,500000,100000,2696
null,10.0,4,null,36047032800,N,N,36047,0.00,F,0,N,3,-3,50,AE,250,N,40.6,null,-74.0,7.7,6.6,N,3,80,1,1988-07-30,2012-09-23,1698,1,2016-09-23,2017-09-23,1,Y,N,NY,11224,2,R,BROOKLYN,N,250000,0,1216


In [34]:
policies = spark.sql('''
          select * from `policies_parquet`
          '''
            )

How many nulls are in each column?

In [36]:
for pol in policies.columns:
  print(pol, "\t", "with null values: ", policies.filter(policies[pol].isNull()).count())

agriculturestructureindicator with null values: 1475814
basefloodelevation with null values: 1520190
basementenclosurecrawlspacetype with null values: 30
cancellationdateoffloodpolicy with null values: 1631622
censustract with null values: 3313
condominiumindicator with null values: 0
construction with null values: 3
countycode with null values: 816
crsdiscount with null values: 0
deductibleamountinbuildingcoverage with null values: 37772
deductibleamountincontentscoverage with null values: 216203
elevatedbuildingindicator with null values: 3
elevationcertificateindicator with null values: 1037195
elevationdifference with null values: 0
federalpolicyfee with null values: 0
floodzone with null values: 17979
hfiaasurcharge with null values: 0
houseofworshipindicator with null values: 1240808
latitude with null values: 3306
locationofcontents with null values: 621875
longitude with null values: 3306
lowestadjacentgrade with null values: 1538078
lowestfloorelevation with null values: 1518635
nonprofitindicator with null values: 1236912
numberoffloorsininsuredbuilding with null values: 17979
obstructiontype with null values: 1525253
occupancytype with null values: 0
originalconstructiondate with null values: 18469
originalnbdate with null values: 0
policycost with null values: 0
policycount with null values: 0
policyeffectivedate with null values: 0
policyterminationdate with null values: 0
policytermindicator with null values: 1
postfirmconstructionindicator with null values: 18468
primaryresidenceindicator with null values: 2
propertystate with null values: 0
reportedzipcode with null values: 0
ratemethod with null values: 63243
regularemergencyprogramindicator with null values: 0
reportedcity with null values: 0
smallbusinessindicatorbuilding with null values: 1187587
totalbuildinginsurancecoverage with null values: 0
totalcontentsinsurancecoverage with null values: 0
totalinsurancepremiumofthepolicy with null values: 0

In [37]:
for pol in policies.columns:
  pol_null_count = policies.filter(policies[pol].isNull()).count()
  if pol_null_count < 1000:
    print(pol, "\t", "with null values: ", pol_null_count)

basementenclosurecrawlspacetype with null values: 30
condominiumindicator with null values: 0
construction with null values: 3
countycode with null values: 816
crsdiscount with null values: 0
elevatedbuildingindicator with null values: 3
elevationdifference with null values: 0
federalpolicyfee with null values: 0
hfiaasurcharge with null values: 0
occupancytype with null values: 0
originalnbdate with null values: 0
policycost with null values: 0
policycount with null values: 0
policyeffectivedate with null values: 0
policyterminationdate with null values: 0
policytermindicator with null values: 1
primaryresidenceindicator with null values: 2
propertystate with null values: 0
reportedzipcode with null values: 0
regularemergencyprogramindicator with null values: 0
reportedcity with null values: 0
totalbuildinginsurancecoverage with null values: 0
totalcontentsinsurancecoverage with null values: 0
totalinsurancepremiumofthepolicy with null values: 0

In [38]:
for col in claims.columns:
  col_null_count = claims.filter(claims[col].isNull()).count()
  if col_null_count < 1000:
    print(col, "\t", "with null values: ", col_null_count)

_c0 with null values: 0
asofdate with null values: 0
basementenclosurecrawlspacetype with null values: 2
reportedcity with null values: 234
policycount with null values: 0
countycode with null values: 272
crsdiscount with null values: 0
dateofloss with null values: 0
elevationdifference with null values: 0
numberoffloorsintheinsuredbuilding with null values: 303
occupancytype with null values: 76
originalnbdate with null values: 0
state with null values: 0
totalbuildinginsurancecoverage with null values: 0
totalcontentsinsurancecoverage with null values: 0
yearofloss with null values: 0
reportedzipcode with null values: 82

In [39]:
policies_df = policies.toPandas()

In [40]:
policies_df.shape

Out[ 5 ]: (1790837, 45)

In [41]:
policies_df.describe()

,agriculturestructureindicator,basefloodelevation,basementenclosurecrawlspacetype,cancellationdateoffloodpolicy,censustract,condominiumindicator,construction,countycode,crsdiscount,deductibleamountinbuildingcoverage,...,primaryresidenceindicator,propertystate,reportedzipcode,ratemethod,regularemergencyprogramindicator,reportedcity,smallbusinessindicatorbuilding,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,totalinsurancepremiumofthepolicy
count,315023,270647,1790807,159215,1787524,1790837,1790834,1790021,1790837,1753065,...,1790835,1790837,1790837,1727594,1790837,1790837,603250,1790837,1790837,1790837
unique,2,3097,5,3701,4598,5,2,64,6,13,...,2,1,1930,17,2,3553,2,4731,1855,13510
top,N,10.0,0,2015-12-18,36103147004,N,N,36059,0.00,1,...,Y,NY,11561,1,R,BROOKLYN,N,250000,100000,390
freq,314866,39999,652372,304,22499,1742595,1788604,508212,1643929,615925,...,1322061,1790837,87968,862515,1790834,101542,596332,1108070,717506,155765


In [42]:
claims_df = claims.toPandas()
claims_df.describe()

,_c0,basefloodelevation,basementenclosurecrawlspacetype,policycount,countycode,crsdiscount,elevationdifference,censustract,latitude,longitude,lowestadjacentgrade,lowestfloorelevation,numberoffloorsintheinsuredbuilding,occupancytype,amountpaidonbuildingclaim,amountpaidoncontentsclaim,amountpaidonincreasedcostofcomplianceclaim,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,yearofloss
count,169312.000000,15655.000000,169310.000000,169312.000000,169040.000000,169312.000000,169312.000000,1.676120e+05,167653.000000,167653.000000,11655.000000,15197.000000,169009.000000,169236.000000,1.645180e+05,110773.000000,89937.000000,1.693120e+05,169312.000000,169312.000000
mean,84655.500000,59.734155,1.132609,1.105391,36070.534087,0.008301,888.857878,3.604282e+10,40.996322,-74.015079,43.474809,166.892143,2.264578,1.339100,2.863880e+04,6210.707788,381.989360,1.610453e+05,27690.436590,2000.843998
std,48876.308726,195.766330,1.196702,3.929104,382.214499,0.029593,313.545474,1.075460e+09,0.743909,1.256027,227.629544,1064.500471,0.816117,0.813504,6.859566e+04,22111.813233,3251.298495,7.024136e+05,47061.344294,13.214408
min,0.000000,-999.000000,0.000000,1.000000,1003.000000,0.000000,-962.000000,3.600300e+04,28.000000,-98.000000,-14.500000,-999.000000,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,1974.000000
25%,42327.750000,7.000000,0.000000,1.000000,36059.000000,0.000000,999.000000,3.605941e+10,40.600000,-74.100000,3.900000,7.000000,2.000000,1.000000,5.426825e+02,0.000000,0.000000,3.300000e+04,0.000000,1989.000000
50%,84655.500000,8.000000,1.000000,1.000000,36081.000000,0.000000,999.000000,3.608109e+10,40.700000,-73.700000,6.700000,9.200000,2.000000,1.000000,6.551130e+03,334.500000,0.000000,1.560000e+05,6200.000000,2010.000000
75%,126983.250000,10.100000,2.000000,1.000000,36103.000000,0.000000,999.000000,3.610312e+10,41.000000,-73.500000,9.000000,12.000000,3.000000,1.000000,3.494000e+04,3718.650000,0.000000,2.500000e+05,37100.000000,2012.000000
max,169311.000000,6666.000000,4.000000,452.000000,54107.000000,0.300000,9998.000000,3.612315e+10,45.000000,-68.700000,9990.000000,9996.000000,6.000000,6.000000,9.467720e+06,500000.000000,45000.000000,1.095000e+08,1000000.000000,2019.000000


Firstly, we're going to select variables which are present in both datasets so we can merge the datasets once we've grouped them. We are also going to remove all the null values from the columns we want to select and lastly, we are going to convert the dataset into a tabular format to merge the two.

In [44]:
policies.groupBy('agriculturestructureindicator', 'condominiumindicator', 'construction', 'primaryresidenceindicator', 'ratemethod', 'regularemergencyprogramindicator').agg(sum("policycount"), count("*")).show()

+-----------------------------+--------------------+------------+-------------------------+----------+--------------------------------+----------------+--------+
agriculturestructureindicator|condominiumindicator|construction|primaryresidenceindicator|ratemethod|regularemergencyprogramindicator|sum(policycount)|count(1)|
+-----------------------------+--------------------+------------+-------------------------+----------+--------------------------------+----------------+--------+
 null| N| N| N| A| R| 583.0| 583|
 null| U| N| Y| W| R| 71.0| 71|
 N| N| N| Y| 6| R| 40.0| 40|
 N| U| Y| N| 1| R| 3.0| 3|
 null| N| N| Y| 7| R| 449858.0| 449858|
 null| N| N| N| E| R| 103.0| 103|
 N| N| Y| N| 1| R| 468.0| 468|
 N| L| N| N| 1| R| 3169.0| 742|
 null| N| N| Y| null| R| 27812.0| 27812|
 null| U| N| Y| 6| R| 9.0| 9|
 N| N| N| Y| 8| R| 141.0| 141|
 N| N| N| Y| W| R| 6280.0| 6280|
 null| N| N| Y| Q| R| 36578.0| 36578|
 null| N| N| N| null| R| 7583.0| 7583|
 null| N| Y| Y| B| R| 9.0| 9|
 N| U| N| Y| 7| R| 2767.0| 2767|
 null| H| N| N| 5| R| 4513.0| 67|
 N| L| N| Y| 2| R| 4.0| 2|
 Y| N| N| N| B| R| 2.0| 2|
 null| U| N| N| P| R| 8.0| 8|
+-----------------------------+--------------------+------------+-------------------------+----------+--------------------------------+----------------+--------+
only showing top 20 rows

In [45]:
claims.groupBy('agriculturestructureindicator', 'condominiumindicator', 'primaryresidence', 'ratemethod').agg(sum("policycount"), count("*"), sum("amountpaidonbuildingclaim"), sum("amountpaidoncontentsclaim") ).show() 

+-----------------------------+--------------------+----------------+----------+----------------+--------+------------------------------+------------------------------+
agriculturestructureindicator|condominiumindicator|primaryresidence|ratemethod|sum(policycount)|count(1)|sum(amountpaidonbuildingclaim)|sum(amountpaidoncontentsclaim)|
+-----------------------------+--------------------+----------------+----------+----------------+--------+------------------------------+------------------------------+
 null| H| Y| 5| 192| 2| 9467720.05| 0.0|
 null| N| Y| 4| 13| 13| 356478.48000000004| 55859.90000000001|
 null| L| N| 2| 126| 20| 1149416.85| null|
 null| N| Y| G| 262| 262| 4397826.7299999995| 631379.69|
 N| N| N| null| 16| 16| 74058.56| 115260.84999999999|
 null| N| N| 6| 1| 1| 0.0| null|
 N| U| Y| 7| 2| 2| 6687.56| 0.0|
 null| N| null| 9| 2| 2| 4180.0| null|
 null| N| Y| Q| 4384| 4384| 2.2382026667E8| 6.3938293210000016E7|
 N| H| Y| 2| 27| 1| 11711.3| null|
 null| N| N| 4| 9| 9| 171851.07| 0.0|
 N| N| N| B| 19| 19| 699783.75| 5944.84|
 null| N| N| A| 40| 40| 1988445.2899999998| 68849.2|
 null| null| null| 3| 6| 6| 0.0| 1685.0|
 null| A| null| 3| 2| 2| 10594.27| 2570.4700000000003|
 null| U| Y| Q| 43| 43| 180858.49999999997| 581083.0|
 N| H| N| 1| 162| 6| 2000.0| null|
 null| N| Y| null| 38| 38| 320384.71| 8490.880000000001|
 N| N| Y| null| 40| 40| 399214.31999999995| 70315.04|
 null| N| N| 2| 495| 495| 2.0641880839999996E7| 1897602.0699999996|
+-----------------------------+--------------------+----------------+----------+----------------+--------+------------------------------+------------------------------+
only showing top 20 rows

In [46]:
claims.groupBy("agriculturestructureindicator").count().orderBy("agriculturestructureindicator").show()

+-----------------------------+------+
agriculturestructureindicator| count|
+-----------------------------+------+
 null|168145|
 N| 1167|
+-----------------------------+------+

In [47]:
claims.columns


Out[ 25 ]: ['_c0',
 'agriculturestructureindicator',
 'asofdate',
 'basefloodelevation',
 'basementenclosurecrawlspacetype',
 'reportedcity',
 'condominiumindicator',
 'policycount',
 'countycode',
 'crsdiscount',
 'dateofloss',
 'elevatedbuildingindicator',
 'elevationcertificateindicator',
 'elevationdifference',
 'censustract',
 'floodzone',
 'houseworship',
 'latitude',
 'locationofcontents',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsintheinsuredbuilding',
 'nonprofitindicator',
 'obstructiontype',
 'occupancytype',
 'originalconstructiondate',
 'originalnbdate',
 'amountpaidonbuildingclaim',
 'amountpaidoncontentsclaim',
 'amountpaidonincreasedcostofcomplianceclaim',
 'postfirmconstructionindicator',
 'ratemethod',
 'smallbusinessindicatorbuilding',
 'state',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'yearofloss',
 'reportedzipcode',
 'primaryresidence']

In [48]:
policies.columns

Out[ 24 ]: ['agriculturestructureindicator',
 'basefloodelevation',
 'basementenclosurecrawlspacetype',
 'cancellationdateoffloodpolicy',
 'censustract',
 'condominiumindicator',
 'construction',
 'countycode',
 'crsdiscount',
 'deductibleamountinbuildingcoverage',
 'deductibleamountincontentscoverage',
 'elevatedbuildingindicator',
 'elevationcertificateindicator',
 'elevationdifference',
 'federalpolicyfee',
 'floodzone',
 'hfiaasurcharge',
 'houseofworshipindicator',
 'latitude',
 'locationofcontents',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'nonprofitindicator',
 'numberoffloorsininsuredbuilding',
 'obstructiontype',
 'occupancytype',
 'originalconstructiondate',
 'originalnbdate',
 'policycost',
 'policycount',
 'policyeffectivedate',
 'policyterminationdate',
 'policytermindicator',
 'postfirmconstructionindicator',
 'primaryresidenceindicator',
 'propertystate',
 'reportedzipcode',
 'ratemethod',
 'regularemergencyprogramindicator',
 'reportedcity',
 'smallbusinessindicatorbuilding',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']

In [49]:
var_list = ['censustract','agriculturestructureindicator', 'condominiumindicator', 'primaryresidence', 'ratemethod', 'basementenclosurecrawlspacetype', 'floodzone', 'reportedzipcode', 'smallbusinessindicatorbuilding','obstructiontype', 'occupancytype']
for var in var_list:
  claims.groupBy(var).count().orderBy(var).show()

+-----------+-----+
censustract|count|
+-----------+-----+
 null| 1700|
 36003| 3|
 36007| 8|
 36009| 3|
 36013| 1|
 36021| 21|
 36025| 2|
 36029| 5|
 36039| 6|
 36059| 4|
 36065| 1|
 36067| 1|
 36069| 1|
 36071| 3|
 36103| 67|
 36105| 3|
 36107| 11|
 36111| 7|
 36119| 2|
36001000000| 1|
+-----------+-----+
only showing top 20 rows

+-----------------------------+------+
agriculturestructureindicator| count|
+-----------------------------+------+
 null|168145|
 N| 1167|
+-----------------------------+------+

+--------------------+------+
condominiumindicator| count|
+--------------------+------+
 A| 124|
 H| 351|
 L| 305|
 N|158517|
 U| 1040|
 unknown| 8975|
+--------------------+------+

+----------------+-----+
primaryresidence|count|
+----------------+-----+
 N|18943|
 Y|84170|
 unknown|66199|
+----------------+-----+

+----------+------+
ratemethod| count|
+----------+------+
 1|127821|
 2| 3530|
 3| 2745|
 4| 42|
 5| 21|
 6| 9|
 7| 18960|
 8| 55|
 9| 18|
 A| 63|
 B| 252|
 E| 1|
 G| 279|
 P| 1693|
 Q| 4747|
 S| 10|
 W| 63|
 unknown| 9003|
+----------+------+

+-------------------------------+-----+
basementenclosurecrawlspacetype|count|
+-------------------------------+-----+
 0|71063|
 1|37602|
 2|35350|
 3|17728|
 4| 7569|
+-------------------------------+-----+

+---------+-----+
floodzone|count|
+---------+-----+
 A| 9481|
 A00| 6|
 A01| 246|
 A02| 783|
 A03| 4854|
 A04| 7520|
 A05| 6768|
 A06| 2549|
 A07| 2973|
 A08| 3045|
 A09| 1851|
 A10| 1495|
 A11| 463|
 A12| 342|
 A13| 129|
 A14| 326|
 A15| 55|
 A16| 97|
 A17| 64|
 A19| 2|
+---------+-----+
only showing top 20 rows

+---------------+-----+
reportedzipcode|count|
+---------------+-----+
 null| 82|
 ,SUF6| 1|
 0| 980|
 0 0| 2|
 0 360| 1|
 00000| 601|
 01 0| 1|
 01196| 2|
 01697| 1|
 02601| 1|
 03120| 1|
 04460| 2|
 06390| 9|
 07001| 1|
 07039| 3|
 07087| 2|
 07662| 2|
 07670| 1|
 07726| 1|
 07778| 1|
+---------------+-----+
only showing top 20 rows

+------------------------------+------+
smallbusinessindicatorbuilding| count|
+------------------------------+------+
 null|167209|
 N| 2078|
 Y| 25|
+------------------------------+------+

+---------------+-----+
obstructiontype|count|
+---------------+-----+
 null|94564|
 *| 4|
 0| 4|
 10|65675|
 15| 736|
 20| 33|
 24| 5|
 30| 74|
 34| 23|
 40| 14|
 50| 3920|
 54| 647|
 60| 2519|
 70| 77|
 80| 1002|
 92| 15|
+---------------+-----+

+-------------+------+
occupancytype| count|
+-------------+------+
 1|137586|
 2| 16930|
 3| 4268|
 4| 10359|
 6| 169|
+-------------+------+

In [50]:
claims = claims.fillna({'condominiumindicator':'unknown', 'primaryresidence':'unknown', 'ratemethod': 'unknown', 'basementenclosurecrawlspacetype':'0', 'floodzone': 'unknown', 'occupancytype':'1', 'amountpaidonbuildingclaim': '0', 'amountpaidoncontentsclaim': '0'})

In [51]:
var_list = ['censustract', 'agriculturestructureindicator', 'condominiumindicator', 'primaryresidenceindicator', 'ratemethod', 'basementenclosurecrawlspacetype', 'floodzone', 'reportedzipcode', 'smallbusinessindicatorbuilding','obstructiontype', 'occupancytype']
for var in var_list:
  policies.groupBy(var).count().orderBy(var).show()

+-----------+-----+
censustract|count|
+-----------+-----+
 null| 3313|
36001000000| 3|
36001000100| 149|
36001000200| 19|
36001000300| 42|
36001000401| 4|
36001000403| 6|
36001000501| 6|
36001000502| 19|
36001000700| 1|
36001000800| 5|
36001001100| 203|
36001001400| 29|
36001001500| 4|
36001001600| 19|
36001001700| 40|
36001001801| 243|
36001001802| 18|
36001001901| 25|
36001001902| 21|
+-----------+-----+
only showing top 20 rows

+-----------------------------+-------+
agriculturestructureindicator| count|
+-----------------------------+-------+
 null|1475814|
 N| 314866|
 Y| 157|
+-----------------------------+-------+

+--------------------+-------+
condominiumindicator| count|
+--------------------+-------+
 A| 100|
 H| 5684|
 L| 5635|
 N|1742595|
 U| 36823|
+--------------------+-------+

+-------------------------+-------+
primaryresidenceindicator| count|
+-------------------------+-------+
 null| 2|
 N| 468774|
 Y|1322061|
+-------------------------+-------+

+----------+------+
ratemethod| count|
+----------+------+
 null| 63243|
 1|862515|
 2| 49816|
 4| 4|
 5| 329|
 6| 262|
 7|666307|
 8| 1585|
 9| 492|
 A| 942|
 B| 55013|
 E| 439|
 F| 20|
 G| 17976|
 P| 11135|
 Q| 41871|
 S| 1120|
 W| 17768|
+----------+------+

+-------------------------------+------+
basementenclosurecrawlspacetype| count|
+-------------------------------+------+
 null| 30|
 0|652372|
 1|496430|
 2|520992|
 3| 49391|
 4| 71622|
+-------------------------------+------+

+---------+-----+
floodzone|count|
+---------+-----+
 null|17979|
 *| 1|
 A|57088|
 A01| 4664|
 A02|11263|
 A03|15672|
 A04|37303|
 A05|37212|
 A06|18228|
 A07|18813|
 A08|19341|
 A09| 8464|
 A10| 6005|
 A11| 3868|
 A12|11170|
 A13| 542|
 A14| 2638|
 A15| 347|
 A16| 453|
 A17| 34|
+---------+-----+
only showing top 20 rows

+---------------+-----+
reportedzipcode|count|
+---------------+-----+
 01159| 5|
 01196| 10|
 01697| 7|
 02601| 9|
 06390| 629|
 06930| 1|
 07039| 9|
 07648| 5|
 07747| 7|
 07778| 6|
 08889| 1|
 10000| 264|
 10001| 410|
 10002| 649|
 10003| 424|
 10004| 725|
 10005| 339|
 10006| 300|
 10007| 214|
 10009| 2403|
+---------------+-----+
only showing top 20 rows

+------------------------------+-------+
smallbusinessindicatorbuilding| count|
+------------------------------+-------+
 null|1187587|
 N| 596332|
 Y| 6918|
+------------------------------+-------+

+---------------+-------+
obstructiontype| count|
+---------------+-------+
 null|1525253|
 *| 1414|
 10| 89634|
 15| 22634|
 20| 415|
 24| 92|
 30| 1207|
 34| 351|
 40| 195|
 50| 68035|
 54| 13129|
 60| 27992|
 70| 2088|
 80| 37858|
 90| 134|
 92| 379|
 94| 6|
 95| 2|
 96| 14|
 98| 5|
+---------------+-------+

+-------------+-------+
occupancytype| count|
+-------------+-------+
 1|1460736|
 2| 160590|
 3| 68703|
 4| 68133|
 6| 32675|
+-------------+-------+

In [52]:
policies = policies.fillna({'condominiumindicator':'unknown', 'primaryresidenceindicator':'unknown', 'ratemethod': 'unknown', 'basementenclosurecrawlspacetype':'0', 'floodzone': 'unknown', 'occupancytype':'1'})

In [53]:
var_list = ['censustract', 'agriculturestructureindicator', 'condominiumindicator', 'primaryresidenceindicator', 'ratemethod', 'basementenclosurecrawlspacetype', 'floodzone', 'reportedzipcode', 'smallbusinessindicatorbuilding','obstructiontype', 'occupancytype']
for var in var_list:
  policies.groupBy(var).count().orderBy(var).show()

+-----------+-----+
censustract|count|
+-----------+-----+
 null| 3313|
36001000000| 3|
36001000100| 149|
36001000200| 19|
36001000300| 42|
36001000401| 4|
36001000403| 6|
36001000501| 6|
36001000502| 19|
36001000700| 1|
36001000800| 5|
36001001100| 203|
36001001400| 29|
36001001500| 4|
36001001600| 19|
36001001700| 40|
36001001801| 243|
36001001802| 18|
36001001901| 25|
36001001902| 21|
+-----------+-----+
only showing top 20 rows

+-----------------------------+-------+
agriculturestructureindicator| count|
+-----------------------------+-------+
 null|1475814|
 N| 314866|
 Y| 157|
+-----------------------------+-------+

+--------------------+-------+
condominiumindicator| count|
+--------------------+-------+
 A| 100|
 H| 5684|
 L| 5635|
 N|1742595|
 U| 36823|
+--------------------+-------+

+-------------------------+-------+
primaryresidenceindicator| count|
+-------------------------+-------+
 N| 468774|
 Y|1322061|
 unknown| 2|
+-------------------------+-------+

+----------+------+
ratemethod| count|
+----------+------+
 1|862515|
 2| 49816|
 4| 4|
 5| 329|
 6| 262|
 7|666307|
 8| 1585|
 9| 492|
 A| 942|
 B| 55013|
 E| 439|
 F| 20|
 G| 17976|
 P| 11135|
 Q| 41871|
 S| 1120|
 W| 17768|
 unknown| 63243|
+----------+------+

+-------------------------------+------+
basementenclosurecrawlspacetype| count|
+-------------------------------+------+
 0|652402|
 1|496430|
 2|520992|
 3| 49391|
 4| 71622|
+-------------------------------+------+

+---------+-----+
floodzone|count|
+---------+-----+
 *| 1|
 A|57088|
 A01| 4664|
 A02|11263|
 A03|15672|
 A04|37303|
 A05|37212|
 A06|18228|
 A07|18813|
 A08|19341|
 A09| 8464|
 A10| 6005|
 A11| 3868|
 A12|11170|
 A13| 542|
 A14| 2638|
 A15| 347|
 A16| 453|
 A17| 34|
 A18| 57|
+---------+-----+
only showing top 20 rows

+---------------+-----+
reportedzipcode|count|
+---------------+-----+
 01159| 5|
 01196| 10|
 01697| 7|
 02601| 9|
 06390| 629|
 06930| 1|
 07039| 9|
 07648| 5|
 07747| 7|
 07778| 6|
 08889| 1|
 10000| 264|
 10001| 410|
 10002| 649|
 10003| 424|
 10004| 725|
 10005| 339|
 10006| 300|
 10007| 214|
 10009| 2403|
+---------------+-----+
only showing top 20 rows

+------------------------------+-------+
smallbusinessindicatorbuilding| count|
+------------------------------+-------+
 null|1187587|
 N| 596332|
 Y| 6918|
+------------------------------+-------+

+---------------+-------+
obstructiontype| count|
+---------------+-------+
 null|1525253|
 *| 1414|
 10| 89634|
 15| 22634|
 20| 415|
 24| 92|
 30| 1207|
 34| 351|
 40| 195|
 50| 68035|
 54| 13129|
 60| 27992|
 70| 2088|
 80| 37858|
 90| 134|
 92| 379|
 94| 6|
 95| 2|
 96| 14|
 98| 5|
+---------------+-------+

+-------------+-------+
occupancytype| count|
+-------------+-------+
 1|1460736|
 2| 160590|
 3| 68703|
 4| 68133|
 6| 32675|
+-------------+-------+

In [54]:
claims_agg = claims.groupBy('condominiumindicator', 'primaryresidence', 'ratemethod', 'basementenclosurecrawlspacetype', 'floodzone', 'occupancytype').agg(sum("policycount"), count("*"), sum("amountpaidonbuildingclaim"), sum("amountpaidoncontentsclaim") )
policy_agg = policies.groupBy('condominiumindicator', 'primaryresidenceindicator', 'ratemethod',  'basementenclosurecrawlspacetype', 'floodzone', 'occupancytype').agg(sum("policycount"), count("*"))

In [55]:
policy_agg.describe().show()

+-------+--------------------+-------------------------+------------------+-------------------------------+---------+-----------------+------------------+-----------------+
summary|condominiumindicator|primaryresidenceindicator| ratemethod|basementenclosurecrawlspacetype|floodzone| occupancytype| sum(policycount)| count(1)|
+-------+--------------------+-------------------------+------------------+-------------------------------+---------+-----------------+------------------+-----------------+
 count| 3824| 3824| 3824| 3824| 3824| 3824| 3824| 3824|
 mean| null| null|2.6369095276220977| 1.5591004184100419| null|2.305439330543933| 536.6213389121339|468.3151150627615|
 stddev| null| null|2.5987641421839998| 1.3402014857284577| null|1.436350956627753|5291.6811368326225|5213.640707506104|
 min| A| N| 1| 0| *| 1| 1.0| 1|
 max| U| unknown| unknown| 4| unknown| 6| 212961.0| 212961|
+-------+--------------------+-------------------------+------------------+-------------------------------+---------+-----------------+------------------+-----------------+

In [56]:
policy_agg.show()

+--------------------+-------------------------+----------+-------------------------------+---------+-------------+----------------+--------+
condominiumindicator|primaryresidenceindicator|ratemethod|basementenclosurecrawlspacetype|floodzone|occupancytype|sum(policycount)|count(1)|
+--------------------+-------------------------+----------+-------------------------------+---------+-------------+----------------+--------+
 N| Y| 1| 0| AE| 1| 133272.0| 133272|
 N| N| B| 4| AE| 1| 482.0| 482|
 N| Y| B| 0| AE| 2| 519.0| 519|
 N| N| 1| 4| AE| 3| 98.0| 98|
 N| Y| unknown| 3| C| 1| 82.0| 82|
 N| Y| 1| 1| X| 3| 153.0| 153|
 N| N| B| 0| AO| 4| 35.0| 35|
 N| N| B| 4| AO| 1| 5.0| 5|
 U| N| 1| 2| B| 3| 12.0| 12|
 N| N| P| 2| C| 2| 17.0| 17|
 U| N| W| 4| AE| 1| 2.0| 2|
 N| N| 1| 4| VE| 6| 6.0| 6|
 N| N| 6| 2| A07| 1| 1.0| 1|
 N| N| B| 2| A11| 1| 1.0| 1|
 N| N| W| 4| A| 6| 3.0| 3|
 N| N| W| 1| A14| 1| 3.0| 3|
 L| N| 1| 0| A10| 2| 18.0| 6|
 N| N| 1| 4| X| 1| 196.0| 196|
 N| N| 2| 4| AE| 2| 413.0| 413|
 N| Y| 1| 0| X| 3| 252.0| 252|
+--------------------+-------------------------+----------+-------------------------------+---------+-------------+----------------+--------+
only showing top 20 rows

In [57]:
claims_agg.show()

+--------------------+----------------+----------+-------------------------------+---------+-------------+----------------+--------+------------------------------+------------------------------+
condominiumindicator|primaryresidence|ratemethod|basementenclosurecrawlspacetype|floodzone|occupancytype|sum(policycount)|count(1)|sum(amountpaidonbuildingclaim)|sum(amountpaidoncontentsclaim)|
+--------------------+----------------+----------+-------------------------------+---------+-------------+----------------+--------+------------------------------+------------------------------+
 N| N| 7| 4| X| 4| 1| 1| 54768.61| 0.0|
 U| N| 7| 1| X| 1| 8| 8| 26275.329999999998| 8042.05|
 U| N| 1| 0| A03| 3| 2| 2| 0.0| 3877.5|
 N| unknown| 1| 2| V08| 4| 2| 2| 267799.12| 0.0|
 N| Y| 1| 3| A08| 2| 1| 1| 73676.09| 0.0|
 N| N| 1| 3| A02| 1| 2| 2| 45632.7| 0.0|
 N| Y| 2| 4| A08| 1| 1| 1| 21338.63| 1997.7|
 N| unknown| 1| 0| A04| 2| 90| 90| 750630.2800000001| 188066.93|
 N| N| 1| 0| AE| 6| 44| 44| 1374488.5899999999| 559820.81|
 unknown| unknown| unknown| 3| A05| 1| 8| 8| 7724.62| 5797.4|
 N| unknown| 1| 3| B| 4| 28| 28| 45901.880000000005| 133045.98999999996|
 unknown| unknown| unknown| 4| C| 2| 3| 3| 702.6| 700.0|
 N| Y| G| 0| unknown| 1| 262| 262| 4397826.7299999995| 631379.69|
 unknown| unknown| unknown| 3| A03| 2| 3| 3| 6948.88| 9499.74|
 unknown| unknown| unknown| 4| A05| 3| 1| 1| 0.0| 1405.0|
 N| Y| 1| 2| A04| 2| 46| 46| 1711361.2999999998| 60386.25|
 N| N| 2| 4| AE| 2| 10| 10| 528388.4199999999| 0.0|
 N| N| 2| 3| A08| 4| 1| 1| 0.0| 0.0|
 U| Y| P| 1| X| 1| 1| 1| 12819.82| 0.0|
 A| unknown| 1| 0| A07| 4| 1| 1| 15000.0| 0.0|
+--------------------+----------------+----------+-------------------------------+---------+-------------+----------------+--------+------------------------------+------------------------------+
only showing top 20 rows

In [58]:
policy_agg.columns

Out[ 62 ]: ['condominiumindicator',
 'primaryresidenceindicator',
 'ratemethod',
 'basementenclosurecrawlspacetype',
 'floodzone',
 'occupancytype',
 'sum(policycount)',
 'count(1)']

In [59]:
policy_agg = policy_agg.withColumnRenamed("primaryresidenceindicator", "primaryresidence")
policy_agg = policy_agg.withColumnRenamed("sum(policycount)", "policycount")
policy_agg = policy_agg.withColumnRenamed("count(1)", "policyindex")
policy_agg.columns

Out[ 96 ]: ['condominiumindicator',
 'primaryresidence',
 'ratemethod',
 'basementenclosurecrawlspacetype',
 'floodzone',
 'occupancytype',
 'policycount',
 'policyindex']

In [60]:
claims_agg.columns

Out[ 94 ]: ['condominiumindicator',
 'primaryresidence',
 'ratemethod',
 'basementenclosurecrawlspacetype',
 'floodzone',
 'occupancytype',
 'policycount',
 'claimsindex',
 'amountpaidonbuildingclaim',
 'amountpaidoncontentsclaim']

In [61]:
claims_agg = claims_agg.withColumnRenamed("sum(amountpaidonbuildingclaim)", "amountpaidonbuildingclaim")
claims_agg = claims_agg.withColumnRenamed("sum(amountpaidoncontentsclaim)", "amountpaidoncontentsclaim")
claims_agg = claims_agg.withColumnRenamed("sum(policycount)", "claimscount")
claims_agg = claims_agg.withColumnRenamed("count(1)", "claimsindex")
claims_agg.columns

Out[ 97 ]: ['condominiumindicator',
 'primaryresidence',
 'ratemethod',
 'basementenclosurecrawlspacetype',
 'floodzone',
 'occupancytype',
 'claimscount',
 'claimsindex',
 'amountpaidonbuildingclaim',
 'amountpaidoncontentsclaim']

In [62]:
for col in claims_agg.columns:
  col_null_count = claims_agg.filter(claims_agg[col].isNull()).count()
  print(col)
  print(col_null_count)

condominiumindicator
0
primaryresidence
0
ratemethod
0
basementenclosurecrawlspacetype
0
floodzone
0
occupancytype
0
policycount
0
claimsindex
0
amountpaidonbuildingclaim
0
amountpaidoncontentsclaim
0

In [63]:
for pol in policy_agg.columns:
  pol_null_count = policy_agg.filter(policy_agg[pol].isNull()).count()
  print(pol)
  print(pol_null_count)

condominiumindicator
0
primaryresidence
0
ratemethod
0
basementenclosurecrawlspacetype
0
floodzone
0
occupancytype
0
policycount
0
policyindex
0

In [64]:
new_df = policy_agg.join(claims_agg, on=['condominiumindicator', 'primaryresidence', 'ratemethod', 'basementenclosurecrawlspacetype', 'floodzone', 'occupancytype' ], how='inner')


In [65]:
new_df.show()

+--------------------+----------------+----------+-------------------------------+---------+-------------+-----------+-----------+-----------+-----------+-------------------------+-------------------------+
condominiumindicator|primaryresidence|ratemethod|basementenclosurecrawlspacetype|floodzone|occupancytype|policycount|policyindex|claimscount|claimsindex|amountpaidonbuildingclaim|amountpaidoncontentsclaim|
+--------------------+----------------+----------+-------------------------------+---------+-------------+-----------+-----------+-----------+-----------+-------------------------+-------------------------+
 N| Y| 1| 0| AE| 1| 133272.0| 133272| 15251| 15251| 8.311218004399998E8| 8.118671117000002E7|
 N| N| B| 4| AE| 1| 482.0| 482| 2| 2| 3900.96| 0.0|
 N| Y| B| 0| AE| 2| 519.0| 519| 1| 1| 2393.51| 2023.78|
 N| Y| 1| 1| X| 3| 153.0| 153| 2| 2| 79078.28| 0.0|
 N| N| B| 0| AO| 4| 35.0| 35| 1| 1| 302500.0| 0.0|
 U| N| 1| 2| B| 3| 12.0| 12| 1| 1| 0.0| 0.0|
 N| N| 1| 4| X| 1| 196.0| 196| 3| 3| 124406.36| 34830.9|
 N| N| 2| 4| AE| 2| 413.0| 413| 10| 10| 528388.4199999999| 0.0|
 N| Y| 1| 0| X| 3| 252.0| 252| 15| 15| 976066.36| 24979.73|
 N| N| 1| 0| A01| 1| 357.0| 357| 9| 9| 123610.58| 5300.0|
 N| N| 2| 2| V11| 1| 20.0| 20| 1| 1| 16587.2| 0.0|
 L| Y| 1| 2| X| 2| 66.0| 18| 4| 1| 0.0| 0.0|
 N| Y| 1| 2| A06| 2| 230.0| 230| 30| 30| 1172405.12| 13513.0|
 N| Y| 1| 0| V11| 1| 246.0| 246| 23| 23| 665666.7400000001| 4664.4|
 N| N| B| 1| A05| 1| 20.0| 20| 1| 1| 0.0| 0.0|
 N| N| 1| 2| AO| 4| 98.0| 98| 12| 12| 205554.62| 21822.77|
 U| N| 2| 1| AE| 2| 18.0| 18| 1| 1| 0.0| 0.0|
 A| N| 2| 2| A05| 4| 2.0| 2| 1| 1| 201609.82| 0.0|
 N| N| 1| 3| A17| 1| 1.0| 1| 1| 1| 107400.0| 29600.0|
 N| N| 7| 4| X| 1| 1490.0| 1490| 14| 14| 443273.42999999993| 67587.01|
+--------------------+----------------+----------+-------------------------------+---------+-------------+-----------+-----------+-----------+-----------+-------------------------+-------------------------+
only showing top 20 rows

In [66]:
print((new_df.count(), len(new_df.columns)))


(1418, 12)

In [67]:
new_df.columns

Out[ 137 ]: ['condominiumindicator',
 'primaryresidence',
 'ratemethod',
 'basementenclosurecrawlspacetype',
 'floodzone',
 'occupancytype',
 'policycount',
 'policyindex',
 'claimscount',
 'claimsindex',
 'amountpaidonbuildingclaim',
 'amountpaidoncontentsclaim']

To build out the loss cost model we will combine two models - severity and frequency model

loss amount/ premium = [loss amount/claims count] * [claims count/premium]

loss cost = severity * frequency

In [70]:
from collections import defaultdict

data_types = defaultdict(list)
for entry in new_df.schema.fields:
    data_types[str(entry.dataType)].append(entry.name)

strings = data_types["StringType"]
print(strings)

['condominiumindicator', 'primaryresidence', 'ratemethod', 'basementenclosurecrawlspacetype', 'floodzone', 'occupancytype']

In [71]:
new_df = new_df.withColumn("claims_ratio", (new_df["amountpaidoncontentsclaim"] + new_df["amountpaidonbuildingclaim"])/new_df["claimscount"])
new_df = new_df.withColumn("inverse_claims_count", new_df["claimscount"]/new_df["policycount"])
new_df = new_df.withColumn("claims_amt", new_df["amountpaidoncontentsclaim"] + new_df["amountpaidonbuildingclaim"])

In [72]:
new_df.select('claims_ratio', 'inverse_claims_count', 'claims_amt', 'claimscount').show()

+------------------+--------------------+-------------------+-----------+
 claims_ratio|inverse_claims_count| claims_amt|claimscount|
+------------------+--------------------+-------------------+-----------+
 59819.58636220575| 0.11443514016447566|9.123085116099999E8| 15251|
 1950.48|0.004149377593360996| 3900.96| 2|
 4417.29|0.001926782273603...| 4417.29| 1|
 39539.14|0.013071895424836602| 79078.28| 2|
 302500.0| 0.02857142857142857| 302500.0| 1|
 0.0| 0.08333333333333333| 0.0| 1|
 53079.08666666667|0.015306122448979591| 159237.26| 3|
 52838.84199999999|0.024213075060532687| 528388.4199999999| 10|
 66736.406| 0.05952380952380952| 1001046.09| 15|
14323.397777777778|0.025210084033613446| 128910.58| 9|
 16587.2| 0.05| 16587.2| 1|
 0.0| 0.06060606060606061| 0.0| 4|
 39530.60400000001| 0.13043478260869565| 1185918.12| 30|
29144.832173913048| 0.09349593495934959| 670331.1400000001| 23|
 0.0| 0.05| 0.0| 1|
18948.115833333333| 0.12244897959183673| 227377.38999999998| 12|
 0.0| 0.05555555555555555| 0.0| 1|
 201609.82| 0.5| 201609.82| 1|
 137000.0| 1.0| 137000.0| 1|
 36490.03142857143|0.009395973154362415| 510860.43999999994| 14|
+------------------+--------------------+-------------------+-----------+
only showing top 20 rows

In [73]:
response_vars = [ 'claimscount', 'claims_amt' ]

variable_list_emblem = ['condominiumindicator',
 'primaryresidence',
 'ratemethod',
 'basementenclosurecrawlspacetype',
 'floodzone',
 'occupancytype']

In [74]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

strings = [var for var in variable_list_emblem if var in data_types["StringType"]]
print(strings)

['condominiumindicator', 'primaryresidence', 'ratemethod', 'basementenclosurecrawlspacetype', 'floodzone', 'occupancytype']

In [75]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

strings = [var for var in variable_list_emblem if var in data_types["StringType"]]
stage_string = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings]
stage_one_hot = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings]

ppl = Pipeline(stages= stage_string + stage_one_hot)
new_df = ppl.fit(new_df).transform(new_df)

In [76]:
new_df.columns

Out[ 145 ]: ['condominiumindicator',
 'primaryresidence',
 'ratemethod',
 'basementenclosurecrawlspacetype',
 'floodzone',
 'occupancytype',
 'policycount',
 'policyindex',
 'claimscount',
 'claimsindex',
 'amountpaidonbuildingclaim',
 'amountpaidoncontentsclaim',
 'claims_ratio',
 'inverse_claims_count',
 'claims_amt',
 'condominiumindicator_string_encoded',
 'primaryresidence_string_encoded',
 'ratemethod_string_encoded',
 'basementenclosurecrawlspacetype_string_encoded',
 'floodzone_string_encoded',
 'occupancytype_string_encoded',
 'condominiumindicator_one_hot',
 'primaryresidence_one_hot',
 'ratemethod_one_hot',
 'basementenclosurecrawlspacetype_one_hot',
 'floodzone_one_hot',
 'occupancytype_one_hot']

In [77]:
categoricals = [var for var in new_df.columns if var.endswith("_one_hot")]
vector_assembler = VectorAssembler(inputCols= categoricals, outputCol= "features")
new_df = vector_assembler.transform(new_df)

training_set, test_set = new_df.randomSplit([0.7, 0.3], seed = 2017)

In [78]:
training_set.show()

+--------------------+----------------+----------+-------------------------------+---------+-------------+-----------+-----------+-----------+-----------+-------------------------+-------------------------+------------------+--------------------+-----------------------------------+-------------------------------+-------------------------+----------------------------------------------+------------------------+----------------------------+----------------------------+------------------------+------------------+---------------------------------------+-----------------+---------------------+--------------------+
condominiumindicator|primaryresidence|ratemethod|basementenclosurecrawlspacetype|floodzone|occupancytype|policycount|policyindex|claimscount|claimsindex|amountpaidonbuildingclaim|amountpaidoncontentsclaim| claims_ratio|inverse_claims_count|condominiumindicator_string_encoded|primaryresidence_string_encoded|ratemethod_string_encoded|basementenclosurecrawlspacetype_string_encoded|floodzone_string_encoded|occupancytype_string_encoded|condominiumindicator_one_hot|primaryresidence_one_hot|ratemethod_one_hot|basementenclosurecrawlspacetype_one_hot|floodzone_one_hot|occupancytype_one_hot| features|
+--------------------+----------------+----------+-------------------------------+---------+-------------+-----------+-----------+-----------+-----------+-------------------------+-------------------------+------------------+--------------------+-----------------------------------+-------------------------------+-------------------------+----------------------------------------------+------------------------+----------------------------+----------------------------+------------------------+------------------+---------------------------------------+-----------------+---------------------+--------------------+
 N| N| B| 0| AO| 4| 35.0| 35| 1| 1| 302500.0| 0.0| 302500.0| 0.02857142857142857| 0.0| 0.0| 3.0| 0.0| 16.0| 3.0| (4,[0],[1.0])| (1,[0],[1.0])| (16,[3],[1.0])| (4,[0],[1.0])| (41,[16],[1.0])| (4,[3],[1.0])|(70,[0,4,8,21,41,...|
 N| Y| 1| 0| AE| 1| 133272.0| 133272| 15251| 15251| 8.311218004399998E8| 8.118671117000002E7| 59819.58636220575| 0.11443514016447566| 0.0| 1.0| 0.0| 0.0| 1.0| 0.0| (4,[0],[1.0])| (1,[],[])| (16,[0],[1.0])| (4,[0],[1.0])| (41,[1],[1.0])| (4,[0],[1.0])|(70,[0,5,21,26,66...|
 N| Y| B| 0| AE| 2| 519.0| 519| 1| 1| 2393.51| 2023.78| 4417.29|0.001926782273603...| 0.0| 1.0| 3.0| 0.0| 1.0| 1.0| (4,[0],[1.0])| (1,[],[])| (16,[3],[1.0])| (4,[0],[1.0])| (41,[1],[1.0])| (4,[1],[1.0])|(70,[0,8,21,26,67...|
 A| N| 2| 2| A05| 4| 2.0| 2| 1| 1| 201609.82| 0.0| 201609.82| 0.5| 4.0| 0.0| 1.0| 1.0| 4.0| 3.0| (4,[],[])| (1,[0],[1.0])| (16,[1],[1.0])| (4,[1],[1.0])| (41,[4],[1.0])| (4,[3],[1.0])|(70,[4,6,22,29,69...|
 L| Y| 1| 2| X| 2| 66.0| 18| 4| 1| 0.0| 0.0| 0.0| 0.06060606060606061| 2.0| 1.0| 0.0| 1.0| 0.0| 1.0| (4,[2],[1.0])| (1,[],[])| (16,[0],[1.0])| (4,[1],[1.0])| (41,[0],[1.0])| (4,[1],[1.0])|(70,[2,5,22,25,67...|
 N| N| 1| 0| A01| 1| 357.0| 357| 9| 9| 123610.58| 5300.0|14323.397777777778|0.025210084033613446| 0.0| 0.0| 0.0| 0.0| 14.0| 0.0| (4,[0],[1.0])| (1,[0],[1.0])| (16,[0],[1.0])| (4,[0],[1.0])| (41,[14],[1.0])| (4,[0],[1.0])|(70,[0,4,5,21,39,...|
 N| N| 1| 4| X| 1| 196.0| 196| 3| 3| 124406.36| 34830.9| 53079.08666666667|0.015306122448979591| 0.0| 0.0| 0.0| 4.0| 0.0| 0.0| (4,[0],[1.0])| (1,[0],[1.0])| (16,[0],[1.0])| (4,[],[])| (41,[0],[1.0])| (4,[0],[1.0])|(70,[0,4,5,25,66]...|
 N| N| 2| 2| V11| 1| 20.0| 20| 1| 1| 16587.2| 0.0| 16587.2| 0.05| 0.0| 0.0| 1.0| 1.0| 21.0| 0.0| (4,[0],[1.0])| (1,[0],[1.0])| (16,[1],[1.0])| (4,[1],[1.0])| (41,[21],[1.0])| (4,[0],[1.0])|(70,[0,4,6,22,46,...|
 N| N| 2| 4| AE| 2| 413.0| 413| 10| 10| 528388.4199999999| 0.0| 52838.84199999999|0.024213075060532687| 0.0| 0.0| 1.0| 4.0| 1.0| 1.0| (4,[0],[1.0])| (1,[0],[1.0])| (16,[1],[1.0])| (4,[],[])| (41,[1],[1.0])| (4,[1],[1.0])|(70,[0,4,6,26,67]...|
 N| N| B| 1| A05| 1| 20.0| 20| 1| 1| 0.0| 0.0| 0.0| 0.05| 0.0| 0.0| 3.0| 2.0| 4.0| 0.0| (4,[0],

In [79]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import GeneralizedLinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [80]:
frequency_claims =  GeneralizedLinearRegression(family="poisson",
                                           link="log",
                                           maxIter=10,
                                           fitIntercept = True,
                                           labelCol = "claimscount",
                                           weightCol = "claims_amt",
                                           regParam=0.3)

frequency_claims.setPredictionCol("prediction_claims_frequency")

Out[ 148 ]: GeneralizedLinearRegression_968fbda77550

In [81]:
severity_claims =  GeneralizedLinearRegression(family="gamma",
                                           link="log",
                                           maxIter=10,
                                           labelCol = "claims_ratio",
                                           weightCol = "inverse_claims_count",
                                           fitIntercept = True,
                                           regParam=0.3)

severity_claims.setPredictionCol("prediction_severity_claims")

Out[ 116 ]: GeneralizedLinearRegression_bb1774b35d97

In [82]:
para_grid = ParamGridBuilder()\
           .addGrid(frequency_claims.regParam, [0.1, 0.3, 0.5, 0.7, 0.9])\
           .build()

evaluator = RegressionEvaluator(labelCol="claimscount",
                                predictionCol="prediction_claims_frequency",
                                metricName="rmse")
cross_val = CrossValidator(estimator = frequency_claims,
                           estimatorParamMaps= para_grid,
                           evaluator = evaluator)

model_frequency = cross_val.fit(training_set)

In [83]:
training_set_sev =  training_set.where((training_set["claims_amt"] > 0))

para_grid = ParamGridBuilder()\
           .addGrid(severity_claims.regParam, [0.1, 0.3, 0.5, 0.7, 0.9])\
           .build()

evaluator = RegressionEvaluator(labelCol="claims_ratio",
                                predictionCol="prediction_severity_claims",
                                metricName="rmse")
cross_val = CrossValidator(estimator = severity_claims,
                           estimatorParamMaps= para_grid,
                           evaluator = evaluator)

model_severity = cross_val.fit(training_set_sev)


Here we will look at how our models perform on the test set and combine the predictions to get our loss cost

In [85]:
df_predictions = model_frequency.transform(test_set)
df_predictions = model_severity.transform(df_predictions)


In [86]:
df_predictions = df_predictions.withColumn("target", df_predictions["claims_amt"] )
df_predictions = df_predictions.withColumn("prediction", df_predictions["prediction_severity_claims"]*df_predictions["prediction_claims_frequency"])


In [87]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf

def shut_down_device(r):
    if r < 0.1:
        return 0.0
    else:
        return r

udf_sd = udf(shut_down_device, DoubleType())

df_predictions = df_predictions.withColumn("truncated_count", udf_sd(df_predictions["prediction_claims_frequency"]))
df_predictions = df_predictions.withColumn("prediction",  df_predictions["prediction_severity_claims"]*df_predictions["truncated_count"])


In [88]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(df_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

from sklearn.metrics import r2_score
prediction_to_pandas = df_predictions.select(["prediction","target"]).toPandas()
r2 = r2_score(prediction_to_pandas.prediction, prediction_to_pandas.target)
print("R-square on test data = %g" % r2)

Root Mean Squared Error (RMSE) on test data = 9.81335e+07
R-square on test data = -1.46816

In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [90]:
display(df_predictions.select('target', 'prediction'))

target,prediction
3900.96,1.2609210495233141E8
79078.28,4.576386376910629E7
0.0,1.5431254897282021E7
227377.38999999998,5.486254026528091E7
137000.0,9.731763537908795E7
0.0,1.2307066565927768E7
510860.43999999994,6.2390063680086285E7
1935562.0399999998,3.552221110992401E7
0.0,2.134602751054956E7
21428.36,3.336800160905544E7


In [91]:
display(df_predictions.select('target'))

target
3900.96
79078.28
0.0
227377.38999999998
137000.0
0.0
510860.43999999994
1935562.0399999998
0.0
21428.36


In [92]:
display(df_predictions.select('prediction'))

prediction
1.2609210495233141E8
4.576386376910629E7
1.5431254897282021E7
5.486254026528091E7
9.731763537908795E7
1.2307066565927768E7
6.2390063680086285E7
3.552221110992401E7
2.134602751054956E7
3.336800160905544E7
